In [1]:
# Dependencies
### Data Cleaning
import csv
import pandas as pd
import numpy as np

### Reflecting DB
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
Base = declarative_base()


In [ ]:
# Read Tab seperated CCES Raw data (using pd.read_csv method) into a pandas Data Frame

file = "data_raw/CCES16_Common_OUTPUT_Feb2018_VV-subset.tab"
file_df = pd.read_csv(file, sep='\t')
file_df.head(2)

In [ ]:
# Read CSV file containing State strings with matching V101 numbers

StatesCSV = 'data_raw/state_names.csv'
StatesDf = pd.read_csv(StatesCSV)
StatesDf.head(2)

In [ ]:
# Merge the two Pandas Data Frames on V101

FinalDF = pd.merge(file_df, StatesDf, on = 'V101')
FinalDF.head(2)

In [ ]:
# Clean the merged Data:

# Drop all rows that have an NA in any column
file_df = FinalDF.dropna()

# Reset the index so there are not missing index values
file_df1 = file_df.reset_index()

# drop second index
file_df1 = file_df1.drop(columns=['index', 'inputstate'])

file_df1.head(2)

In [ ]:
# Rename columns with more descriptive titles

file_dfRename = file_df1.rename(columns = {
    'CC16_330a':'GunBackgroundChecks_16',
    'CC16_330b':'ProhibitPublication_16',
    'CC16_330d':'BanAssultWeapons_16',
    'CC16_330e':'MakeCCPEasier_16',
    'CC16_332a':'AlwaysAllowChoice_16',
    'CC16_332b':'RapeIncestorHealth_16',
    'CC16_332c':'ProhibitMoreThan20Weeks_16',
    'CC16_332d':'EmployersDeclineBenefits_16',
    'CC16_332e':'ProhibitFedFunds_16',
    'CC16_335': 'GayMarriage'
})

# Reorder the columns

CCES_df = file_dfRename[['V101', 'lookupzip', 'StateName', 'countyfips', 'birthyr', 'gender', 'educ',
       'GunBackgroundChecks_16', 'ProhibitPublication_16',
       'BanAssultWeapons_16', 'MakeCCPEasier_16', 'AlwaysAllowChoice_16',
       'RapeIncestorHealth_16', 'ProhibitMoreThan20Weeks_16',
       'EmployersDeclineBenefits_16', 'ProhibitFedFunds_16', 'GayMarriage']]
CCES_df.head(2)

In [ ]:
########## Convert some of the columns from number answers to string value responses

# Convert gender column
CCES_df['gender'] = CCES_df['gender'].apply(lambda x: 'male' if x== 1 else 'female')

# map and convert edu column
CCES_df['educ'] = CCES_df['educ'].map({
                                            1: 'NoHS',
                                            2: 'HS_Grad',
                                            3: 'Some College',
                                            4: '2-year',
                                            5: '4-year',
                                            6: 'Post-Grad'
                                          })

# CC16_330a 1 support, 2 oppose, 8 skipped / Gun Background Checks_16
CCES_df['GunBackgroundChecks_16'] = CCES_df['GunBackgroundChecks_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330b 1 support, 2 oppose, 8 skipped  'CC16_330b':'Prohibit Publication_16',
CCES_df['ProhibitPublication_16'] = CCES_df['ProhibitPublication_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330d 1 support, 2 oppose, 8 skipped 'CC16_330d':'Ban Assult Weapons_16',
CCES_df['BanAssultWeapons_16'] = CCES_df['BanAssultWeapons_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330e 1 support, 2 oppose, 8 skipped 'CC16_330e':'Make CCP Easier_16',
CCES_df['MakeCCPEasier_16'] = CCES_df['MakeCCPEasier_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332a 1 support, 2 oppose, 8 skipped 'CC16_332a':'Always Allow Choice_16',
CCES_df['AlwaysAllowChoice_16'] = CCES_df['AlwaysAllowChoice_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332b 1 support, 2 oppose, 8 skipped   'CC16_332b':'Rape, Incest, or Health_16',
CCES_df['RapeIncestorHealth_16'] = CCES_df['RapeIncestorHealth_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332c 1 support, 2 oppose, 8 skipped 'CC16_332c':'Prohibit MoreThan20 Weeks_16',
CCES_df['ProhibitMoreThan20Weeks_16'] = CCES_df['ProhibitMoreThan20Weeks_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332d 1 support, 2 oppose, 8 skipped   'CC16_332d':'Employers decline benefits_16',
CCES_df['EmployersDeclineBenefits_16'] = CCES_df['EmployersDeclineBenefits_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332e 1 support, 2 oppose, 8 skipped  'CC16_332e':'Prohibit Fed Funds_16',
CCES_df['ProhibitFedFunds_16'] = CCES_df['ProhibitFedFunds_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_335 1 support, 2 oppose, 8 skipped  'CC16_335': 'GayMarriage'
CCES_df['GayMarriage'] = CCES_df['GayMarriage'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})
CCES_df.head(2)

In [ ]:
## Troubleshooting to add up the total responses for each column
## If the responses for each section show a difference value of 0 - that's good news!

GunBcSupport = (len(CCES_df[CCES_df['GunBackgroundChecks_16']=='Support']))
GunBCOppose = (len(CCES_df[CCES_df['GunBackgroundChecks_16']=='Oppose']))
GunBcTotal = (len(CCES_df[CCES_df['GunBackgroundChecks_16']!='none']))
print('Gun Support Total Support: ', GunBcSupport)
print('Total Oppose: ', GunBCOppose)
print('Total Rows: ', GunBcTotal)
print('Total Difference', (GunBcTotal - GunBCOppose - GunBcSupport))
print('--------------------------------------------------------------------------')

ProhibitPubSupport = (len(CCES_df[CCES_df['ProhibitPublication_16']=='Support']))
ProhibitPubOppose = (len(CCES_df[CCES_df['ProhibitPublication_16']=='Oppose']))
ProhibitPubTotal = (len(CCES_df[CCES_df['ProhibitPublication_16']!='none']))
print('Prohibit Publication_16: ', ProhibitPubSupport)
print('Total Oppose: ', ProhibitPubOppose)
print('Total Rows: ', ProhibitPubTotal)
print('Total Difference', (ProhibitPubTotal - ProhibitPubOppose - ProhibitPubSupport))
print('--------------------------------------------------------------------------')
BanAssultSupport = (len(CCES_df[CCES_df['BanAssultWeapons_16']=='Support']))
BanAssultOppose = (len(CCES_df[CCES_df['BanAssultWeapons_16']=='Oppose']))
BanAssultTotal = (len(CCES_df[CCES_df['BanAssultWeapons_16']!='none']))
print('Gun Support Total Support: ', BanAssultSupport)
print('Total Oppose: ', BanAssultOppose)
print('Total Rows: ', BanAssultTotal)
print('Total Difference', (BanAssultTotal - BanAssultOppose - BanAssultSupport))
print('--------------------------------------------------------------------------')
#########################################################################################
CCPSupport = (len(CCES_df[CCES_df['MakeCCPEasier_16']=='Support']))
CCPOppose = (len(CCES_df[CCES_df['MakeCCPEasier_16']=='Oppose']))
CCPTotal = (len(CCES_df[CCES_df['MakeCCPEasier_16']!='none']))
print('Gun Support Total Support: ', CCPSupport)
print('Total Oppose: ', CCPOppose)
print('Total Rows: ', CCPTotal)
print('Total Difference', (CCPTotal - CCPOppose - CCPSupport))
print('--------------------------------------------------------------------------')

AlwaysSupport = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']=='Support']))
AlwaysOppose = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']=='Oppose']))
AlwaysTotal = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']!='none']))
print('Prohibit Publication_16: ', AlwaysSupport)
print('Total Oppose: ', AlwaysOppose)
print('Total Rows: ', AlwaysTotal)
print('Total Difference', (AlwaysTotal - AlwaysOppose - AlwaysSupport))
print('--------------------------------------------------------------------------')
HealthSupport = (len(CCES_df[CCES_df['RapeIncestorHealth_16']=='Support']))
HealthOppose = (len(CCES_df[CCES_df['RapeIncestorHealth_16']=='Oppose']))
HealthTotal = (len(CCES_df[CCES_df['RapeIncestorHealth_16']!='none']))
print('Gun Support Total Support: ', HealthSupport)
print('Total Oppose: ', HealthOppose)
print('Total Rows: ', HealthTotal)
print('Total Difference', (HealthTotal - HealthOppose - HealthSupport))
print('--------------------------------------------------------------------------')

#########################################################################################
MoreThan20Support = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']=='Support']))
MoreThan20Oppose = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']=='Oppose']))
MoreThan20Total = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']!='none']))
print('Gun Support Total Support: ', MoreThan20Support)
print('Total Oppose: ', MoreThan20Oppose)
print('Total Rows: ', MoreThan20Total)
print('Total Difference', (MoreThan20Total - MoreThan20Oppose - MoreThan20Support))
print('--------------------------------------------------------------------------')

DeclineBenSupport = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']=='Support']))
DeclineBenOppose = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']=='Oppose']))
DeclineBenTotal = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']!='none']))
print('Prohibit Publication_16: ', DeclineBenSupport)
print('Total Oppose: ', DeclineBenOppose)
print('Total Rows: ', DeclineBenTotal)
print('Total Difference', (DeclineBenTotal - DeclineBenOppose - DeclineBenSupport))
print('--------------------------------------------------------------------------')
NoFedFundsSupport = (len(CCES_df[CCES_df['ProhibitFedFunds_16']=='Support']))
NoFedFundsOppose = (len(CCES_df[CCES_df['ProhibitFedFunds_16']=='Oppose']))
NoFedFundsTotal = (len(CCES_df[CCES_df['ProhibitFedFunds_16']!='none']))
print('Gun Support Total Support: ', NoFedFundsSupport)
print('Total Oppose: ', NoFedFundsOppose)
print('Total Rows: ', NoFedFundsTotal)
print('Total Difference', (NoFedFundsTotal - NoFedFundsOppose - NoFedFundsSupport))
print('--------------------------------------------------------------------------')

#########################################################################################
GaySupport = (len(CCES_df[CCES_df['GayMarriage']=='Support']))
GayOppose = (len(CCES_df[CCES_df['GayMarriage']=='Oppose']))
GayTotal = (len(CCES_df[CCES_df['GayMarriage']!='none']))
print('GayMarriage Total Support: ', GaySupport)
print('Total Oppose: ', GayOppose)
print('Total Rows: ', GayTotal)
print('Total Difference', (GayTotal - GayOppose - GaySupport))
print('--------------------------------------------------------------------------')

GenderMale = (len(CCES_df[CCES_df['gender']=='male']))
GenderFemale = (len(CCES_df[CCES_df['gender']=='female']))
GenderTotal = (len(CCES_df[CCES_df['gender']!='none']))
print('gender: ', GenderMale)
print('Total Male: ', GenderMale)
print('Total Female: ', GenderFemale)
print('Total Rows: ', GenderTotal)
print('Total Difference', (GenderTotal - GenderFemale - GenderMale))
print('--------------------------------------------------------------------------')

# Education

NoHS = (len(CCES_df[CCES_df['educ']=='NoHS']))
HS_Grad = (len(CCES_df[CCES_df['educ']=='HS_Grad']))
SomeCollege = (len(CCES_df[CCES_df['educ']=='Some College']))
year2 = (len(CCES_df[CCES_df['educ']=='2-year']))
year4= (len(CCES_df[CCES_df['educ']=='4-year']))
PostGrad = (len(CCES_df[CCES_df['educ']=='Post-Grad']))
eduTotal = (len(CCES_df[CCES_df['educ']!='none']))
print('educ Total Support: ' )
print('Total NoHS: ',NoHS )
print('Total HS_Grad: ', HS_Grad)
print('Total SomeCollege: ', SomeCollege)
print('Total year2: ', year2)
print('Total year4: ', year4)
print('Total PostGrad: ', PostGrad)
print('Total Rows: ', eduTotal)
print('Total Difference', (eduTotal - NoHS - HS_Grad- SomeCollege - year2 - year4 - PostGrad ))
print('--------------------------------------------------------------------------')


In [ ]:
# Create the engine to connect to a sqlite file.
# The file will not be created until the class, base, and metadata are executed

engine = create_engine("sqlite:///db/CCES_Ver50.sqlite")

# Create a variable to manage session.queries later on...
session = Session(engine)

In [ ]:
# Display Dataframe Column types to ensure the class is created correctly

CCES_df.dtypes

In [ ]:
### Define our Survey Class for the sqlite db table


class Survey(Base):
    __tablename__ = 'CCES_16'
    index = Column(Integer, primary_key=True)
    V101 = Column(Integer)
    lookupzip = Column(Integer)
    StateName = Column(String(255))
    countyfips = Column(Float)
  
    birthyr = Column(Integer)
    gender = Column(String(255))
    educ = Column(String(255))
   
    GunBackgroundChecks_16 = Column(String(255))
    ProhibitPublication_16 = Column(String(255))
    BanAssultWeapons_16 = Column(String(255))
    MakeCCPEasier_16 = Column(String(255))
    AlwaysAllowChoice_16 = Column(String(255))
    RapeIncestorHealth_16 = Column(String(255))
    ProhibitMoreThan20Weeks_16 = Column(String(255))
    Employersdeclinebenefits_16 = Column(String(255))
    ProhibitFedFunds_16 = Column(String(255))
    GayMarriage = Column(String(255))

In [ ]:
# Create the CCES_16 table within the database
# This step actually creates the sqlite file

Base.metadata.create_all(engine)

In [ ]:
# Loop to transfer/write the DF data into the sqlite file.
# The only way I could get this to work was in a loop where the notebook wasn't trying to write more than 50 rows at a time.
# I believe this is a hardware limitation of my laptop where it could not process more than 50 rows at a time in the local
# memory without the notebook becoming unresponsive. This method has worked accuratly and quickly for 10+ consecutive tests.

# It takes approximately 1 minute for this cell to run.

x1 = 0
x2 = 50

while x1 < 62881:
    
    tempDF = CCES_df.iloc[x1:x2]
    tempDF.to_sql('CCES_16', con=engine, if_exists='append')
    x1 += 50
    x2 += 50
    print('Round complete, starting', x1, 'to', x2)
    
    

In [ ]:
# Declare a base using automap_base()
Base = automap_base()
Base

In [ ]:
# Use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [ ]:
# Print all of the classses mapped to the Base (this provides a list of the table names in the sqlite.db)
Base.classes.keys()

In [ ]:
# Assign the CCES_16 class to a variable called 'CCES'
CCES = Base.classes.CCES_16

In [ ]:
### Accuracy Test 1
# Query support for gay marriage from pandas DF and from Sqlite file

ApproveGay = session.query(CCES).filter(CCES.GayMarriage == 'Support').count()
print(f'Sqlite query of votes supporting Gay Marrige: {ApproveGay}')
print(f'Pandas DF Sort of votes supporing Gay Marriage: {GaySupport}')

In [ ]:
### Accuracy Test 2
# Query support for Gun Background Checks from pandas DF and Sqlite file
SupportChecks = session.query(CCES).filter(CCES.GunBackgroundChecks_16 == 'Support').count()
print(f'Sqlite support votes: {SupportChecks}')
print(f'Pandas DF support votes: {GunBcSupport}')

In [ ]:
### Multi Filter Test 1
# Query Males who support background checks
# Query Females who support background checks
# Total number of support background check votes in Pandas

MaleCheckSupport = session.query(CCES).filter(CCES.gender == 'male').filter(CCES.GunBackgroundChecks_16 == 'Support').count()
FemaleCheckSupport = session.query(CCES).filter(CCES.gender == 'female').filter(CCES.GunBackgroundChecks_16 == 'Support').count()

print(f'Query Male = {MaleCheckSupport}')
print(f'Query Female = {FemaleCheckSupport}')
print('Query total = ', MaleCheckSupport + FemaleCheckSupport)
print(f'Total Support = {GunBcSupport}')

In [1]:
# Filter by state how many respondents were supportive of Gun Background Checks

StatesFor = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).group_by(CCES.StateName).all()
StatesFor[0:5]

NameError: name 'session' is not defined